In [10]:
## read from a text file. maybe ask user to enter the name of their text file and remind them to put it in the correct folder?
## put content from text file into a variable named "puzzle". it should be a 2d array

In [5]:
import os

def read_sudoku_from_file(directory, file_name):
    file_path = os.path.join(directory, file_name)
    
    puzzle = [[0 for _ in range(9)] for _ in range(9)]  # Create a 9x9 grid of zeros
    with open(file_path, 'r') as file:
        for row_idx, line in enumerate(file):
            # Only process lines that are part of the puzzle (ignore empty lines or other text)
            if row_idx < 9:
                for col_idx, char in enumerate(line.strip()):
                    # If the character is '*', it represents an empty cell
                    if char == '*':
                        puzzle[row_idx][col_idx] = 0
                    # Otherwise, if the character is a digit and not greater than '9', add it to the puzzle
                    elif char.isdigit() and int(char) <= 9:
                        puzzle[row_idx][col_idx] = int(char)
    return puzzle

# Example usage
current_directory = os.getcwd()
file_name = "puzzle.txt"
given_puzzle = read_sudoku_from_file(current_directory, file_name)

if given_puzzle:
    print("Given puzzle:")
    for row in given_puzzle:
        print(row)
else:
    print("No puzzle exists.")


# Define Sudoku solver function
def solve_sudoku(grid):# Implement backtracking algorithm
    # Find an empty cell
    empty_cell = find_empty_cell(grid)
    if not empty_cell:
        return True  # Puzzle solved

    row, col = empty_cell

    # Try placing digits from 1 to 9
    for num in range(1, 10):
        if is_valid_move(grid, row, col, num):
            grid[row][col] = num

            # Recursively solve the rest of the puzzle
            if solve_sudoku(grid):
                return True  # Solution found

            # Backtrack
            grid[row][col] = 0

    return False  # No solution found


def find_empty_cell(grid):
    # Find the first empty cell (cell with value 0)
    for row in range(9):
        for col in range(9):
            if grid[row][col] == 0:
                return (row, col)
    return None  # No empty cell found


def is_valid_move(grid, row, col, num):
    # Check if num is not already in the current row, column, or subgrid
    return (
        not used_in_row(grid, row, num)
        and not used_in_col(grid, col, num)
        and not used_in_subgrid(grid, row - row % 3, col - col % 3, num)
    )


def used_in_row(grid, row, num):
    return num in grid[row]


def used_in_col(grid, col, num):
    return any(row[col] == num for row in grid)


def used_in_subgrid(grid, start_row, start_col, num):
    for row in range(3):
        for col in range(3):
            if grid[row + start_row][col + start_col] == num:
                return True
    return False

# Example usage
current_directory = os.getcwd()
puzzle_path = "puzzle.txt"
solved_puzzle = read_sudoku_from_file(current_directory, file_name)

if solve_sudoku(solved_puzzle):
    print("Solution:")
    for row in solved_puzzle:
        print(row)
else:
    print("No solution exists.")

Given puzzle:
[8, 2, 1, 0, 0, 0, 0, 0, 7]
[0, 0, 0, 8, 0, 0, 0, 6, 0]
[0, 6, 0, 9, 3, 0, 0, 0, 5]
[0, 0, 8, 2, 0, 1, 6, 0, 0]
[0, 0, 0, 7, 0, 0, 2, 8, 4]
[2, 4, 0, 6, 0, 3, 7, 0, 0]
[6, 0, 5, 0, 0, 0, 1, 0, 3]
[0, 7, 0, 0, 5, 0, 0, 0, 0]
[9, 1, 2, 0, 0, 0, 0, 0, 6]
Solution:
[8, 2, 1, 5, 6, 4, 3, 9, 7]
[5, 9, 3, 8, 1, 7, 4, 6, 2]
[4, 6, 7, 9, 3, 2, 8, 1, 5]
[7, 5, 8, 2, 4, 1, 6, 3, 9]
[1, 3, 6, 7, 9, 5, 2, 8, 4]
[2, 4, 9, 6, 8, 3, 7, 5, 1]
[6, 8, 5, 4, 2, 9, 1, 7, 3]
[3, 7, 4, 1, 5, 6, 9, 2, 8]
[9, 1, 2, 3, 7, 8, 5, 4, 6]


In [10]:
import tkinter as tk
from tkinter import filedialog

# Initialize the main application window
app = tk.Tk()
app.title("Sudoku Solver")
app.configure(bg='white')  # Set background to white for minimalism

GRID_SIZE = 9
CELL_SIZE = 50
FONT = ('Century Gothic', 14)  # Choose a modern, readable font
app.geometry(f"{2 * GRID_SIZE * CELL_SIZE + 50}x{GRID_SIZE * CELL_SIZE + 150}")

# Function to update the displayed Sudoku grids
def update_grids(given_puzzle, solved_puzzle):
    for widget in before_frame.winfo_children():
        widget.destroy()
    for widget in after_frame.winfo_children():
        widget.destroy()

    # For the 'Before' grid, pass given_puzzle for both the puzzle and given_puzzle arguments
    create_grid(before_frame, given_puzzle, given_puzzle, "Before", 1)

    # For the 'After' grid, pass solved_puzzle for the puzzle argument and given_puzzle for the given_puzzle argument
    create_grid(after_frame, solved_puzzle, given_puzzle, "After", 1)


def load_puzzle():
    filepath = filedialog.askopenfilename(filetypes=[("Text files", "*.txt")])
    if not filepath:
        return
    global given_puzzle, solved_puzzle
    given_puzzle = read_sudoku_from_file(os.path.dirname(filepath), os.path.basename(filepath))
    solved_puzzle = [row[:] for row in given_puzzle]
    if solve_sudoku(solved_puzzle):
        update_grids(given_puzzle, solved_puzzle)
    else:
        print("No solution exists.")

load_button = tk.Button(app, text="Load New Puzzle", command=load_puzzle, font=FONT, bg='lightgray', fg='black')
load_button.grid(row=0, columnspan=2, pady=(10, 20))

before_frame = tk.Frame(app, bg='white')
before_frame.grid(row=1, column=0, padx=25, pady=25)

after_frame = tk.Frame(app, bg='white')
after_frame.grid(row=1, column=1, padx=25, pady=25)

def create_grid(frame, puzzle, given_puzzle, title, start_row):
    title_label = tk.Label(frame, text=title, font=FONT, bg='white', fg='black')
    title_label.grid(row=start_row, columnspan=GRID_SIZE)
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            cell_frame = tk.Frame(frame, bg='white', highlightbackground='black', highlightcolor='black', highlightthickness=1, width=CELL_SIZE, height=CELL_SIZE)
            cell_frame.grid(row=i + start_row + 1, column=j)
            cell_frame.pack_propagate(False)

            cell_text = puzzle[i][j] if puzzle[i][j] != 0 else ""
            # Determine the color based on whether the cell was filled by the solver or was part of the original puzzle
            if puzzle[i][j] == given_puzzle[i][j] and puzzle[i][j] != 0:
                cell_color = 'black'  # Color for original puzzle numbers
            else:
                cell_color = 'green'  # Color for numbers filled in by the solver

            cell_label = tk.Label(cell_frame, text=cell_text, bg='white', fg=cell_color, font=FONT)
            cell_label.pack(expand=True, fill='both')

update_grids(given_puzzle, solved_puzzle)

app.mainloop()
